# Amazon Bedrock AgentCore 런타임(Runtime)에서 Amazon Bedrock 모델을 사용한 LangGraph 에이전트(Agent) 호스팅

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임(Runtime)을 사용하여 기존 에이전트(Agent)를 호스팅하는 방법을 배웁니다.

Amazon Bedrock 모델을 사용하는 LangGraph 예제에 초점을 맞춥니다. Amazon Bedrock 모델을 사용하는 Strands Agents는 [여기](../01-strands-with-bedrock-model)를,
OpenAI 모델을 사용하는 Strands Agents는 [여기](../03-strands-with-openai-model)를 참조하세요.

### 튜토리얼 상세 정보

| 항목                | 상세 내용                                                                    |
|:--------------------|:-----------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형(Conversational)                                                       |
| 에이전트 유형       | 단일(Single)                                                                 |
| 에이전틱 프레임워크 | LangGraph                                                                    |
| LLM 모델            | Anthropic Claude Haiku 4.5                                                   |
| 튜토리얼 구성요소   | AgentCore 런타임(Runtime)에서 에이전트(Agent) 호스팅. LangGraph와 Amazon Bedrock 모델 사용 |
| 튜토리얼 분야       | 범용(Cross-vertical)                                                         |
| 예제 난이도         | 쉬움(Easy)                                                                   |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK 및 boto3                                  |

### 튜토리얼 아키텍처

이 튜토리얼에서는 기존 에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deploy)하는 방법을 설명합니다.

데모 목적으로 Amazon Bedrock 모델을 사용하는 LangGraph 에이전트(Agent)를 사용합니다.

이 예제에서는 `get_weather`와 `get_time` 두 가지 도구(Tool)를 가진 매우 간단한 에이전트(Agent)를 사용합니다.

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore 런타임(Runtime)에서 에이전트(Agent) 호스팅
* Amazon Bedrock 모델 사용
* LangGraph 사용

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명(Credentials)
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker 실행 중

In [1]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## 에이전트(Agent) 생성 및 로컬 실험

에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deploy)하기 전에, 실험 목적으로 로컬에서 개발하고 실행해 보겠습니다.

프로덕션 에이전틱(Agentic) 애플리케이션의 경우 에이전트(Agent) 생성 프로세스와 호출(Invocation) 프로세스를 분리해야 합니다. AgentCore 런타임(Runtime)에서는 에이전트(Agent)의 호출(Invocation) 부분을 `@app.entrypoint` 데코레이터로 장식하고 이를 런타임(Runtime)의 진입점(Entry Point)으로 사용합니다. 먼저 실험 단계에서 각 에이전트(Agent)가 어떻게 개발되는지 살펴보겠습니다.

여기서 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [2]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
import argparse
import json
import operator
import math

# Create calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Define safe functions that can be used in expressions
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # Math functions
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # Evaluate the expression safely
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom weather tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"

# Define the agent using manual LangGraph construction
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

def langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = langgraph_bedrock(json.loads(args.payload))
    print(response)

Writing langgraph_bedrock.py


#### 로컬 에이전트(Agent) 호출(Invoke)

In [3]:
!python langgraph_bedrock.py '{"prompt": "What is the weather now?"}'

/Users/seokju/amazon-bedrock-agentcore-samples/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
The weather right now is **sunny**! ☀️ It's a nice day out there.


## AgentCore 런타임(Runtime) 배포(Deployment)를 위한 에이전트(Agent) 준비

이제 에이전트(Agent)를 AgentCore 런타임(Runtime)에 배포(Deploy)해 보겠습니다. 이를 위해 다음을 수행해야 합니다:
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp`으로 런타임(Runtime) 앱 가져오기
* 코드에서 `app = BedrockAgentCoreApp()`으로 앱 초기화
* 호출(Invocation) 함수를 `@app.entrypoint` 데코레이터로 장식
* `app.run()`으로 AgentCore 런타임(Runtime)이 에이전트(Agent) 실행을 제어하도록 설정

### Amazon Bedrock 모델을 사용한 LangGraph
Amazon Bedrock 모델을 사용하는 LangGraph부터 시작하겠습니다. 다른 프레임워크(Framework)와 모델을 사용하는 다른 예제는 상위 디렉토리에서 확인할 수 있습니다.

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import operator
import math

app = BedrockAgentCoreApp()

# Create calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Define safe functions that can be used in expressions
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # Math functions
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # Evaluate the expression safely
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom weather tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"

# Define the agent using manual LangGraph construction
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

@app.entrypoint
def langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()

## 내부에서 어떤 일이 일어나나요?

`BedrockAgentCoreApp`을 사용하면 자동으로 다음이 수행됩니다:

* 포트 8080에서 수신 대기하는 HTTP 서버 생성
* 에이전트(Agent) 요청 처리를 위한 필수 `/invocations` 엔드포인트(Endpoint) 구현
* 헬스 체크(Health Check)를 위한 `/ping` 엔드포인트(Endpoint) 구현 (비동기 에이전트(Agent)에 매우 중요)
* 적절한 콘텐츠 타입 및 응답 형식 처리
* AWS 표준에 따른 오류 처리 관리

## AgentCore 런타임(Runtime)에 에이전트(Agent) 배포(Deploy)

`CreateAgentRuntime` 작업은 컨테이너(Container) 이미지, 환경 변수 및 암호화 설정을 지정할 수 있는 포괄적인 구성(Configuration) 옵션을 지원합니다. 또한 프로토콜(Protocol) 설정(HTTP, MCP)과 인가(Authorization) 메커니즘을 구성하여 클라이언트가 에이전트(Agent)와 통신하는 방식을 제어할 수 있습니다.

**참고:** 운영 모범 사례는 코드를 컨테이너(Container)로 패키징하고 CI/CD 파이프라인(Pipeline)과 IaC를 사용하여 ECR에 푸시하는 것입니다.

이 튜토리얼에서는 Amazon Bedrock AgentCore Python SDK를 사용하여 아티팩트(Artifact)를 쉽게 패키징하고 AgentCore 런타임(Runtime)에 배포(Deploy)합니다.

### AgentCore 런타임(Runtime) 배포(Deployment) 구성(Configure)

먼저 스타터 툴킷(Starter Toolkit)을 사용하여 진입점(Entrypoint), 방금 생성한 실행 역할(Execution Role) 및 requirements 파일로 AgentCore 런타임(Runtime) 배포(Deployment)를 구성(Configure)합니다. 또한 시작 시 Amazon ECR 리포지토리(Repository)를 자동 생성하도록 스타터 킷을 구성(Configure)합니다.

구성(Configure) 단계에서 애플리케이션 코드를 기반으로 Docker 파일이 생성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "langgraph_claude_getting_started"
response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### AgentCore 런타임(Runtime)에 에이전트(Agent) 시작(Launch)

Docker 파일이 준비되었으니 에이전트(Agent)를 AgentCore 런타임(Runtime)에 시작(Launch)하겠습니다. 이렇게 하면 Amazon ECR 리포지토리(Repository)와 AgentCore 런타임(Runtime)이 생성됩니다.

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore 런타임(Runtime) 상태 확인
AgentCore 런타임(Runtime)을 배포(Deploy)했으니 배포(Deployment) 상태를 확인해 보겠습니다.

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore 런타임(Runtime) 호출(Invoke)

마지막으로 페이로드(Payload)를 사용하여 AgentCore 런타임(Runtime)을 호출(Invoke)할 수 있습니다.

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How much is 2+2?"})
invoke_response

### 호출(Invocation) 결과 처리

이제 호출(Invocation) 결과를 처리하여 애플리케이션에 포함할 수 있습니다.

In [ ]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### boto3를 사용한 AgentCore 런타임(Runtime) 호출(Invoke)

AgentCore 런타임(Runtime)이 생성되었으므로 모든 AWS SDK로 호출(Invoke)할 수 있습니다. 예를 들어 boto3의 `invoke_agent_runtime` 메서드를 사용할 수 있습니다.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather now?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## 정리(Cleanup) (선택 사항)

생성된 AgentCore 런타임(Runtime)을 정리(Cleanup)하겠습니다.

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# 축하합니다!